In [1]:
import os
import time
import boto3
import logging

In [2]:
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger('mxnet_data')
fh = logging.FileHandler('data-publishing.log')
fh.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

In [3]:
def find_last(text, token):
    position = []
    for n in range(len(text)):
        if text.find(token, n, n+1) != -1:
            position.append(n)
    return position[len(position)-1]

In [4]:
def publish_to_queue(sqs, queuename, filename):
    data_file = open(filename, 'r')
    data_list = data_file.readlines()
    data_file.close()    
    message = ''
    messageDeduplicationId=filename[find_last(filename, '_')+1:find_last(filename, '.')]
    messageGroupId=filename[find_last(filename, '/')+1:find_last(filename, '_')]
    for record in data_list:
        message = message + record
    queue = sqs.get_queue_by_name(QueueName=queuename)
    response = queue.send_message(
        MessageBody=message,
        MessageDeduplicationId=messageDeduplicationId,
        MessageGroupId=messageGroupId
    )
    return response

In [5]:
sqs = boto3.resource('sqs', 'us-east-1')

In [6]:
queue = 'training-data-queue.fifo'
logger.info('Publishing Queue - ' + queue)

In [7]:
lines_per_file = 120
smallfile = None
progress = 0
if not os.path.exists('./dataset/incremental'):
    os.makedirs('./dataset/incremental') 
if os.path.exists('check.dat'):
    check = open('check.dat','r')
    progress = int(check.readlines()[0])
    check.close()
with open('./dataset/train.csv') as bigfile:
    for lineno, line in enumerate(bigfile):
        if int((lineno - 1) / lines_per_file) < progress:
            continue
        if (lineno - 1)  % lines_per_file == 0:
            if smallfile:
                smallfile.close()
                print("Created file : " + smallfile.name)
                response = publish_to_queue(sqs,queue,small_filename)
                logger.info('Data Packet - ' + small_filename[find_last(small_filename, '/')+1:find_last(small_filename, '.')]+ ' published at ' + response['ResponseMetadata']['HTTPHeaders']['date'])               
                check = open('check.dat','w') 
                check.write(str(int(small_filename[find_last(small_filename, '_')+1:find_last(small_filename, '.')])))
                check.close()
                time.sleep(30)                
            small_filename = './dataset/incremental/train_{}.csv'.format(str(int((lineno - 1) / lines_per_file) + 1).zfill(3))
            print(small_filename)
            smallfile = open(small_filename, "w")
        if line.find("pixel") < 0:
            smallfile.write(line)
    if smallfile:
        smallfile.close()
if os.path.exists('check.dat'):
    os.remove('check.dat')

./dataset/incremental/train_053.csv
Created file : ./dataset/incremental/train_053.csv


KeyboardInterrupt: 